# Ejercicio 12 : Asistente RAG Recipes

## Objetivo de la práctica

Construir un asistente que:

1. Recibe una pregunta del usuario
2. Recupera texto relevante de un corpus (ej. libro de Baeza-Yates)
3. Genera una respuesta basada en los documentos encontrados



## Parte 0: Librerías necesarias
- openai
- faiss-cpu
- sentence-transformers

In [1]:
import fitz  
import pandas as pd
import os
import re
import json
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from openai import OpenAI
from openai import OpenAI

## Parte 1: Carga del corpus

Aquí se debe cargar el corpus con las recetas del json

In [2]:
import json
import pandas as pd

corpus_path = '../data/recipes_data.json'
data = []

with open(corpus_path, 'r', encoding='utf-8') as file:
    recipes = json.load(file)
    for url, recipe in recipes.items():
        title = recipe.get('title', 'Sin título')
        content = "\n".join(recipe.get('steps', []))  # Unimos los pasos como contenido
        data.append({
            'url': url,
            'title': title,
            'content': content
        })

df = pd.DataFrame(data)
df


,url,title,content
0,https://www.allrecipes.com/recipe/93168/rotiss...,Rotisserie Chicken,Gather all ingredients. Preheat an outdoor gri...
1,https://www.allrecipes.com/recipe/93168/rotiss...,Rotisserie Chicken,Gather all ingredients. Preheat an outdoor gri...
2,https://www.allrecipes.com/recipe/238575/cilan...,Cilantro-Lime Grilled Chicken,"Whisk cilantro, lime juice, garlic salt, and b..."
3,https://www.allrecipes.com/recipe/275062/butte...,Buttermilk Barbecue Chicken,"Whisk buttermilk, brown sugar, cider vinegar, ..."
4,https://www.allrecipes.com/recipe/274724/grill...,Grilled Spatchcocked Chicken,Place salt in a large bowl or Dutch oven; add ...
5,https://www.allrecipes.com/recipe/14531/beer-b...,Beer Butt Chicken,Preheat an outdoor grill for low heat and ligh...
6,https://www.allrecipes.com/recipe/221093/good-...,Good Frickin' Paprika Chicken,"Whisk together yogurt, garlic, 3 tablespoons p..."
7,https://www.allrecipes.com/recipe/264278/miso-...,Miso Honey Chicken,Combine miso and honey in a bowl. Pour in rice...
8,https://www.allrecipes.com/recipe/258659/rosem...,Rosemary Buttermilk Chicken,"Mix buttermilk, garlic, paprika, salt, and pep..."
9,https://www.allrecipes.com/recipe/222936/smoke...,Smoked Beer Butt Chicken,Preheat grill for medium heat and lightly oil ...


## Parte 2: Procesamiento del Corpus

Aquí se debe obtener el corpus procesado. El corpus estará formado por documentos que corresponden a las secciones (o subsecciones) de los libros. Cada documento debe indicar a qué libro corresponde, así como las páginas en las que está dentro de ese libro.

Recuerden que los documentos procesados no deben contener textos o caracteres ajenos al tema del que tratan.  

In [3]:
all_toc = []

with open(corpus_path, 'r', encoding='utf-8') as file:
    recipes = json.load(file)

    for url, recipe in recipes.items():
        title = recipe.get("title", "Sin título").strip()
        resume = recipe.get("resume", "")
        ingredients = recipe.get("ingredients", [])
        steps = recipe.get("steps", [])
        nutrition_facts = recipe.get("nutrition_facts", {})
        img_url = recipe.get("image_url", "")

        # Sección 1: Resumen
        all_toc.append({
            "receta": title,
            "nivel": 1,
            "sección": "1",
            "título": "Resumen",
            "texto": resume,
            "fuente_TOC": "JSON"
        })

        # Sección 2: Ingredientes
        ingredients_text = "\n".join(f"- {ing}" for ing in ingredients)
        all_toc.append({
            "receta": title,
            "nivel": 1,
            "sección": "2",
            "título": "Ingredientes",
            "texto": ingredients_text,
            "fuente_TOC": "JSON"
        })

        # Sección 3: Pasos
        for idx, step in enumerate(steps, 1):
            all_toc.append({
                "receta": title,
                "nivel": 2,
                "sección": f"3.{idx}",
                "título": f"Paso {idx}",
                "texto": step,
                "fuente_TOC": "JSON"
            })

        # Sección 4: Información nutricional
        nutrition_text = "\n".join(f"{k}: {v}" for k, v in nutrition_facts.items())
        all_toc.append({
            "receta": title,
            "nivel": 1,
            "sección": "4",
            "título": "Información Nutricional",
            "texto": nutrition_text,
            "fuente_TOC": "JSON"
        })

        # Sección 5: Imagen (como URL)
        all_toc.append({
            "receta": title,
            "nivel": 1,
            "sección": "5",
            "título": "Imagen",
            "texto": img_url,
            "fuente_TOC": "JSON"
        })

# Convertir a DataFrame
df_toc_mejorado = pd.DataFrame(all_toc)
df_toc_mejorado

,receta,nivel,sección,título,texto,fuente_TOC
0,Rotisserie Chicken,1,1,Resumen,This rotisserie chicken recipe is so easy to m...,JSON
1,Rotisserie Chicken,1,2,Ingredientes,- 1 (3 pound) whole chicken\n- 1 pinch salt\n-...,JSON
2,Rotisserie Chicken,2,3.1,Paso 1,Gather all ingredients. Preheat an outdoor gri...,JSON
3,Rotisserie Chicken,2,3.2,Paso 2,Season chicken cavity with a pinch of salt. Ti...,JSON
4,Rotisserie Chicken,2,3.3,Paso 3,Place rotisserie over the preheated grill and ...,JSON
...,...,...,...,...,...,...
163,Beer Can Chicken,2,3.3,Paso 3,Rinse chicken under cold running water. Discar...,JSON
164,Beer Can Chicken,2,3.4,Paso 4,"Place chicken, standing on the can, directly o...",JSON
165,Beer Can Chicken,2,3.5,Paso 5,Remove chicken from the grill and discard beer...,JSON
166,Beer Can Chicken,1,4,Información Nutricional,Calories: 546\nFat: 27g\nCarbs: 24g\nProtein: 48g,JSON


In [4]:
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Reemplazar múltiples espacios por uno
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación y caracteres especiales
    return text.strip()
df_toc_mejorado['texto_normalizado'] = df_toc_mejorado['texto'].apply(normalize_text)
df_toc_mejorado

,receta,nivel,sección,título,texto,fuente_TOC,texto_normalizado
0,Rotisserie Chicken,1,1,Resumen,This rotisserie chicken recipe is so easy to m...,JSON,this rotisserie chicken recipe is so easy to m...
1,Rotisserie Chicken,1,2,Ingredientes,- 1 (3 pound) whole chicken\n- 1 pinch salt\n-...,JSON,1 3 pound whole chicken 1 pinch salt ¼ cup b...
2,Rotisserie Chicken,2,3.1,Paso 1,Gather all ingredients. Preheat an outdoor gri...,JSON,gather all ingredients preheat an outdoor gril...
3,Rotisserie Chicken,2,3.2,Paso 2,Season chicken cavity with a pinch of salt. Ti...,JSON,season chicken cavity with a pinch of salt tie...
4,Rotisserie Chicken,2,3.3,Paso 3,Place rotisserie over the preheated grill and ...,JSON,place rotisserie over the preheated grill and ...
...,...,...,...,...,...,...,...
163,Beer Can Chicken,2,3.3,Paso 3,Rinse chicken under cold running water. Discar...,JSON,rinse chicken under cold running water discard...
164,Beer Can Chicken,2,3.4,Paso 4,"Place chicken, standing on the can, directly o...",JSON,place chicken standing on the can directly on ...
165,Beer Can Chicken,2,3.5,Paso 5,Remove chicken from the grill and discard beer...,JSON,remove chicken from the grill and discard beer...
166,Beer Can Chicken,1,4,Información Nutricional,Calories: 546\nFat: 27g\nCarbs: 24g\nProtein: 48g,JSON,calories 546 fat 27g carbs 24g protein 48g


In [5]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Aplicar al DataFrame de TOC
df_toc_mejorado['texto_filtrado'] = df_toc_mejorado['texto_normalizado'].apply(remove_stopwords)
df_toc_mejorado

,receta,nivel,sección,título,texto,fuente_TOC,texto_normalizado,texto_filtrado
0,Rotisserie Chicken,1,1,Resumen,This rotisserie chicken recipe is so easy to m...,JSON,this rotisserie chicken recipe is so easy to m...,rotisserie chicken recipe easy make simple sea...
1,Rotisserie Chicken,1,2,Ingredientes,- 1 (3 pound) whole chicken\n- 1 pinch salt\n-...,JSON,1 3 pound whole chicken 1 pinch salt ¼ cup b...,1 3 pound whole chicken 1 pinch salt ¼ cup but...
2,Rotisserie Chicken,2,3.1,Paso 1,Gather all ingredients. Preheat an outdoor gri...,JSON,gather all ingredients preheat an outdoor gril...,gather ingredients preheat outdoor grill high ...
3,Rotisserie Chicken,2,3.2,Paso 2,Season chicken cavity with a pinch of salt. Ti...,JSON,season chicken cavity with a pinch of salt tie...,season chicken cavity pinch salt tie legs toge...
4,Rotisserie Chicken,2,3.3,Paso 3,Place rotisserie over the preheated grill and ...,JSON,place rotisserie over the preheated grill and ...,place rotisserie preheated grill cook 10 minut...
...,...,...,...,...,...,...,...,...
163,Beer Can Chicken,2,3.3,Paso 3,Rinse chicken under cold running water. Discar...,JSON,rinse chicken under cold running water discard...,rinse chicken cold running water discard gible...
164,Beer Can Chicken,2,3.4,Paso 4,"Place chicken, standing on the can, directly o...",JSON,place chicken standing on the can directly on ...,place chicken standing directly preheated gril...
165,Beer Can Chicken,2,3.5,Paso 5,Remove chicken from the grill and discard beer...,JSON,remove chicken from the grill and discard beer...,remove chicken grill discard beer cover chicke...
166,Beer Can Chicken,1,4,Información Nutricional,Calories: 546\nFat: 27g\nCarbs: 24g\nProtein: 48g,JSON,calories 546 fat 27g carbs 24g protein 48g,calories 546 fat 27g carbs 24g protein 48g


## Parte 3: Cálculo de Embeddings e Indexación en base de datos vectorial

Aquí, una vez que se ha calculado el embedding de cada documento, se deberá indexar este embedding en una base de datos vectorial como FAISS, ChromaDB o Pinecone

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')
#generar funcion embeddings para todo el texto
def generate_embeddings(text):
    return model.encode(text)
# Aplicar la función de embeddings al DataFrame
df_toc_mejorado['embeddings'] = df_toc_mejorado['texto_normalizado'].apply(generate_embeddings)
df_toc_mejorado

,receta,nivel,sección,título,texto,fuente_TOC,texto_normalizado,texto_filtrado,embeddings
0,Rotisserie Chicken,1,1,Resumen,This rotisserie chicken recipe is so easy to m...,JSON,this rotisserie chicken recipe is so easy to m...,rotisserie chicken recipe easy make simple sea...,"[-0.023811527, -0.059269447, -0.014561937, -0...."
1,Rotisserie Chicken,1,2,Ingredientes,- 1 (3 pound) whole chicken\n- 1 pinch salt\n-...,JSON,1 3 pound whole chicken 1 pinch salt ¼ cup b...,1 3 pound whole chicken 1 pinch salt ¼ cup but...,"[-0.048987847, -0.042561907, 0.020795476, 0.04..."
2,Rotisserie Chicken,2,3.1,Paso 1,Gather all ingredients. Preheat an outdoor gri...,JSON,gather all ingredients preheat an outdoor gril...,gather ingredients preheat outdoor grill high ...,"[-0.012102308, -0.0099021215, 0.028517244, 0.0..."
3,Rotisserie Chicken,2,3.2,Paso 2,Season chicken cavity with a pinch of salt. Ti...,JSON,season chicken cavity with a pinch of salt tie...,season chicken cavity pinch salt tie legs toge...,"[0.021088272, 0.0021674812, 0.0018654675, 0.00..."
4,Rotisserie Chicken,2,3.3,Paso 3,Place rotisserie over the preheated grill and ...,JSON,place rotisserie over the preheated grill and ...,place rotisserie preheated grill cook 10 minut...,"[0.046412326, 0.01917596, -0.016095642, 0.0264..."
...,...,...,...,...,...,...,...,...,...
163,Beer Can Chicken,2,3.3,Paso 3,Rinse chicken under cold running water. Discar...,JSON,rinse chicken under cold running water discard...,rinse chicken cold running water discard gible...,"[-0.03429031, -0.05328498, 0.011423018, -0.047..."
164,Beer Can Chicken,2,3.4,Paso 4,"Place chicken, standing on the can, directly o...",JSON,place chicken standing on the can directly on ...,place chicken standing directly preheated gril...,"[-0.015713332, 0.007325911, -0.09648346, 0.004..."
165,Beer Can Chicken,2,3.5,Paso 5,Remove chicken from the grill and discard beer...,JSON,remove chicken from the grill and discard beer...,remove chicken grill discard beer cover chicke...,"[-0.032796048, 0.053265978, -0.020818694, 0.06..."
166,Beer Can Chicken,1,4,Información Nutricional,Calories: 546\nFat: 27g\nCarbs: 24g\nProtein: 48g,JSON,calories 546 fat 27g carbs 24g protein 48g,calories 546 fat 27g carbs 24g protein 48g,"[-0.004715458, -0.00885202, -0.0130152395, 0.0..."


In [7]:
#sacar los embeddings del df
embeddings = np.array(df_toc_mejorado['embeddings'].tolist())
embeddings

array([[-0.02381153, -0.05926945, -0.01456194, ...,  0.05301848,
         0.00985585,  0.0397049 ],
       [-0.04898785, -0.04256191,  0.02079548, ...,  0.0187341 ,
        -0.08399854,  0.07460677],
       [-0.01210231, -0.00990212,  0.02851724, ...,  0.02661788,
        -0.08309534, -0.04186901],
       ...,
       [-0.03279605,  0.05326598, -0.02081869, ...,  0.01544208,
        -0.04566492,  0.04807663],
       [-0.00471546, -0.00885202, -0.01301524, ..., -0.04609126,
         0.00216587, -0.06349945],
       [ 0.01473399,  0.02866226, -0.05085005, ...,  0.06327105,
         0.05968732,  0.00757135]], shape=(168, 384), dtype=float32)

In [8]:
# Crear un índice FAISS para búsqueda de similitud
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)  

## Parte 4: Búsqueda y obtención del contexto

En esta parte debemos definir una _query_ y buscar los documentos que más se relacionan con ella.

Estos documentos formarán el contexto que vamos a entregar al LLM.

In [9]:
# Solicitar input del usuario
query = input("Ingrese su consulta: ").lower()

In [10]:
print("Consulta ingresada:", query)
query_embedding = model.encode(query)
query_embedding

Consulta ingresada: chicken


array([-4.19756435e-02, -1.32821109e-02, -3.84519994e-02,  4.70161363e-02,
       -4.22213897e-02,  9.95706953e-03,  1.65552616e-01, -8.08368064e-03,
        7.46925622e-02, -4.19138111e-02, -1.35424052e-04, -4.13294770e-02,
       -2.45801006e-02, -2.45018415e-02, -2.68594064e-02, -6.07909635e-02,
        5.55869006e-02, -6.74602240e-02, -3.80850062e-02, -9.65465009e-02,
       -1.52622491e-01,  2.72846804e-03,  4.57986593e-02,  1.00333965e-03,
       -3.66761684e-02,  6.42383918e-02,  2.05617640e-02,  3.25725265e-02,
       -2.97653694e-02, -8.06936324e-02,  3.97887193e-02, -9.64505598e-03,
       -2.50394951e-04, -2.05167476e-03, -4.87854332e-02, -1.74947008e-02,
        3.42604076e-03, -4.67465855e-02,  7.17530549e-02,  6.85145427e-03,
        1.77733358e-02, -8.47468451e-02,  6.91125318e-02, -4.46848050e-02,
        4.89137694e-02,  4.59626056e-02,  1.33782895e-02,  5.71032278e-02,
        5.12409657e-02, -3.11464556e-02, -3.81297953e-02, -2.83781253e-02,
       -7.45173544e-02,  

In [11]:
# Realizar la búsqueda de similitud
k = 5  # Número de resultados a recuperar
D, I = index.search(np.array([query_embedding]), k)  # D: distancias, I: índices
#guardar en top_k_indices los índices de los k resultados más cercanos
top_k_indices = I[0]
top_k_indices

array([98, 83, 80,  3, 13])

In [12]:
# Mostrar los resultados de búsqueda semántica
for i, idx in enumerate(I[0]):
    fila = df_toc_mejorado.iloc[idx]
    print(f"Resultado {i + 1}:")
    print(f"Receta: {fila['receta']}")
    print(f"Sección: {fila['sección']} - {fila['título']}")
    print(f"Texto: {fila['texto_filtrado'][:200]}...")  # Solo primeros 200 caracteres
    print(f"Distancia: {D[0][i]:.4f}")
    print("-" * 50)


Resultado 1:
Receta: The Best Beer Can Chicken Ever
Sección: 1 - Resumen
Texto: spicy skin tender roast chicken...
Distancia: 0.9180
--------------------------------------------------
Resultado 2:
Receta: Rosemary Buttermilk Chicken
Sección: 1 - Resumen
Texto: summertime chickengrilling favorite family find making week week...
Distancia: 0.9242
--------------------------------------------------
Resultado 3:
Receta: Miso Honey Chicken
Sección: 3.7 - Paso 7
Texto: plate chicken alongside lemon wedges sprinkled cayenne pepper squeeze spicy lemon chicken cut...
Distancia: 0.9309
--------------------------------------------------
Resultado 4:
Receta: Rotisserie Chicken
Sección: 3.2 - Paso 2
Texto: season chicken cavity pinch salt tie legs together kitchen string tie wings bird secure chicken rotisserie attachment dotdash meredith food studios...
Distancia: 0.9684
--------------------------------------------------
Resultado 5:
Receta: Rotisserie Chicken
Sección: 3.2 - Paso 2
Texto: season ch

In [13]:
# Agrupar fragmentos por receta (usamos los top 3 más similares)
recetas_encontradas = df_toc_mejorado.iloc[top_k_indices[:3]]['receta'].unique()

# Extraer toda la información de esas recetas
context = ""
for receta in recetas_encontradas:
    partes = df_toc_mejorado[df_toc_mejorado['receta'] == receta]
    context += f"\n\nReceta: {receta}\n"
    for _, fila in partes.iterrows():
        context += f"\n{fila['título']}:\n{fila['texto']}\n"

context

"\n\nReceta: The Best Beer Can Chicken Ever\n\nResumen:\nSpicy skin and tender roast chicken!\n\nIngredientes:\n- 1 cup chocolate stout beer\n- 3  green Thai chile peppers\n- 3 cloves garlic, peeled\n- 3 tablespoons brown sugar\n- 2 teaspoons dry mustard\n- 1 teaspoon garam masala\n- 1 teaspoon kosher salt\n- 1 teaspoon ground black pepper\n- ½ teaspoon cayenne pepper\n- ½ teaspoon ground cumin\n- ½ teaspoon ground cinnamon\n- ½ teaspoon onion powder\n- ½ teaspoon garlic powder\n- ¼ teaspoon ground nutmeg\n- 1 (5 pound) whole chicken\n\nPaso 1:\nPreheat grill for medium heat. If using charcoal, push coals to the side of the grilling area for indirect heat.\n\nPaso 2:\nPour stout beer into an empty 12-ounce soda can and drop Thai chilies and garlic cloves into the can. Mix brown sugar, dry mustard, garam masala, kosher salt, black pepper, cayenne pepper, cumin, cinnamon, onion powder, garlic powder, and nutmeg in a bowl.\n\nPaso 3:\nRinse the chicken and coat the skin with the entire ba

## Parte 5: Generación de Respuesta

Aquí, entregamos el contexto al LLM, y él nos responde a la _query_ con una explicación en lenguaje natural.

In [14]:
prompt = f"""Eres una aplicación de Recuperación Aumentada con Generación (RAG) que responde siempre en español. A continuación se te proporciona información de recetas. Tu tarea es:
1. Identificar cuál receta es más relevante para la solicitud del usuario.
2. Generar la **receta completa** con un formato de presentación limpio, humano y legible.
IMPORTANTE: 
No devuelvas la receta en formato JSON ni en bloque de código. Preséntala como una receta común, organizada por secciones, así:

Título: ...
Calificación: ...
Tiempo total: ...
Porciones: ...

Ingredientes:
- ingrediente 1
- ingrediente 2

Pasos:
1. Paso uno
2. Paso dos

Información nutricional:
- Calorías: ...
- Grasas: ...
- Proteínas: ...

Si no puedes generar la receta, responde: "No tengo suficiente información para generar esa receta."

Contexto:
{context}

Pregunta:
Genera la receta completa relacionada con: {query}
"""

In [ ]:
#aqui api key de openai

In [38]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

# Guardar el resultado
respuesta = completion.choices[0].message.content

# Limpiar bloque Markdown si es necesario
import re
respuesta_limpia = re.sub(r"^```[a-z]*|```$", "", respuesta.strip(), flags=re.MULTILINE)

print("Receta generada:\n")
print(respuesta_limpia)


Receta generada:

Título: Pollo Miso y Miel

Calificación: ★★★★☆

Tiempo total: 2 a 12 horas (incluyendo marinado) + 40 minutos de cocción

Porciones: 4

Ingredientes:
- 3 cucharadas de miso blanco
- 2 cucharadas de miel
- ¼ de taza de vinagre de arroz
- 2 cucharaditas de salsa picante
- 1 cucharada de sal kosher
- 1 pollo entero, cortado por la mitad, con puntas de alas separadas
- Sal kosher al gusto
- ½ limón, cortado en cuñas, o al gusto
- 1 pizca de cayena

Pasos:
1. En un bol, combina el miso y la miel. Agrega el vinagre de arroz, seguido de la salsa picante y 1 cucharada de sal. Mezcla bien hasta obtener una consistencia mayormente suave.
   
2. Haz 3 a 4 cortes en las piernas y los muslos de las mitades del pollo. Mezcla el pollo en el adobo hasta que esté bien cubierto. Refrigera durante 2 a 12 horas.

3. Gira el pollo en el bol. Espolvorea más sal kosher por encima.

4. Precalienta una parrilla a 175 grados C (350 grados F) a fuego indirecto.

5. Coloca el pollo en la parrill